In [3]:
import pandas as pd

df = pd.read_parquet('/home/nikl/programming/fhdw/knowledge_engineering_ausarbeitung/fakenews_detection/data/model_evaluation/with_peft/distilbert-base-uncased/summary_logs.parquet')
display(df)

,epoch,step,total_flos,train_loss,train_runtime,train_samples_per_second,train_steps_per_second
0,1.0,1380,5.989150e+15,0.406191,795.2602,55.52,1.735
